<center>
    <p style="text-align:center">
    <img alt="arize logo" src="https://storage.googleapis.com/arize-assets/arize-logo-white.jpg" width="300"/>
        <br>
        <a href="https://docs.arize.com/arize/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/client_python">GitHub</a>
        |
        <a href="https://join.slack.com/t/arize-ai/shared_invite/zt-1px8dcmlf-fmThhDFD_V_48oU7ALan4Q">Community</a>
    </p>
</center>

<center><h1>Using Arize with Langchain 🦜🔗 </h1></center>

LangChain is a framework that helps you prototype LLM applications quickly. Use Arize and LangChain together to trace, evaluate, and iterate on your LLM apps and agents.

### Running This Notebook
1. Step through each section below, pressing play on the code blocks to run the cells.
2. Log in your browser to the Arize App
3. Copy and paste your Arize Space ID and API key

# Step 0. Install Dependencies

Install Langchain and Arize packages.

⚠️ Use a GPU to save time generating embeddings. Click on 'Runtime', select 'Change Runtime Type' and
select 'GPU'.

In [ ]:
!pip3 install -q langchain langchain_community langchain-openai 'httpx<0.28'
!pip3 install -q arize arize-otel openinference-instrumentation-langchain 
!pip3 install -q 'arize[AutoEmbeddings]'

# Step 1. Setup Tracing

Copy the Arize `API_KEY` and `SPACE_ID` from your Space Settings page (shown below) to the variables in the cell below.

<center><img src="https://storage.googleapis.com/arize-assets/barcelos/Screenshot%202024-11-11%20at%209.28.27%E2%80%AFPM.png" width="700"></center>



In [ ]:
# Import open-telemetry dependencies
from arize.otel import register
from getpass import getpass

# Setup OTEL via our convenience function
tracer_provider = register(
    space_id=getpass("Enter your Arize Space ID:"),
    api_key=getpass("Enter your Arize API Key:"),
    project_name="langchain-tracing",
)

# Import the automatic instrumentor from OpenInference
from openinference.instrumentation.langchain import LangChainInstrumentor

# Finish automatic instrumentation
LangChainInstrumentor().instrument(tracer_provider=tracer_provider)

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = getpass("Your OpenAI API Key: ")

## Step 2: Define LLM

In [ ]:
from langchain_openai import OpenAI

llm = OpenAI(temperature=0, verbose=True)

## Step 3: Test LLM Responses and Logging into Arize
Use some simple prompts to test if the LLM works properly and each prompt-response pair is logged into Arize with embeddings.

In [ ]:
llm_result = llm.generate(
    [
        "Tell me an interesting fact about pandas.",
        "Explain the concept of overfitting in 2 sentences.",
    ]
)

## Step 4: Test LLM Chain and Agents with Arize Integration

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain

template = """You are a playwright. Given the title of play, it is your job to write a synopsis for that title.
Title: {title}
Playwright: This is a synopsis for the above play:"""
prompt_template = PromptTemplate(input_variables=["title"], template=template)
synopsis_chain = LLMChain(llm=llm, prompt=prompt_template)

template = """You are a play critic from the New York Times. Given the synopsis of play, it is your job to write a review for that play.
Play Synopsis:
{synopsis}
Review from a New York Times play critic of the above play:"""
prompt_template = PromptTemplate(
    input_variables=["synopsis"], template=template
)
review_chain = LLMChain(llm=llm, prompt=prompt_template)

overall_chain = SimpleSequentialChain(
    chains=[synopsis_chain, review_chain], verbose=True
)

test_prompts = [
    {
        "input": "documentary about pandas who are about be extinct because of global warming"
    },
    {"input": "once upon a time in hollywood"},
    {"input": "the best mo observability tooling"},
]
overall_chain.apply(test_prompts)